# INST326 Week 7 Lecture
## Error Handling and Testing Fundamentals

**Duration:** 75 minutes (1:15)  
**Date:** October 27, 2025

---

## Learning Objectives
By the end of this lecture, you will be able to:
- Implement exception handling using try/except/else/finally blocks
- Raise exceptions appropriately to signal error conditions
- Create custom exception classes for domain-specific errors
- Write basic unit tests using Python's unittest framework
- Apply error handling to validate garden system data
- Understand the difference between catching vs. raising exceptions

---
# Part 1: Exception Handling Fundamentals
## (35 minutes)

### What Are Exceptions?

**Exceptions** are Python's way of handling errors and unexpected situations. When something goes wrong, Python "raises" an exception. If we don't handle it, our program crashes.

**Why This Matters for Garden Management:**
- Invalid container dimensions (negative length)
- Impossible planting dates (planting tomatoes before last frost)
- Incompatible plant combinations (planting enemies together)
- File reading errors (missing plant database)

**Key Insight:** Good error handling makes your program robust and user-friendly instead of just crashing with cryptic messages.

### Basic try/except Structure

The most basic error handling pattern:

```python
try:
    # Code that might cause an error
    risky_operation()
except SomeError:
    # What to do if that error occurs
    handle_error()
```

Let's see this in action with garden data:

In [6]:
# Example 1: Converting user input to numbers
def get_container_depth(depth_string):
    """Convert user input to container depth in inches.

    Args:
        depth_string: String from user input

    Returns:
        float: Depth in inches
    """
    try:
        depth = float(depth_string)
        return depth
    except ValueError:
        print(f"'{depth_string}' is not a valid number. Using default depth of 12 inches.")
        return 12.0

# Test it out
print(get_container_depth("8.5"))      # Works: 8.5
print(get_container_depth("eight"))    # Handles error: 12.0

8.5
'eight' is not a valid number. Using default depth of 12 inches.
12.0


In [7]:
# Example 2: Catching multiple exception types
def calculate_plants_per_row(row_length, plant_spacing):
    """Calculate how many plants fit in a row.

    Args:
        row_length: Length of row in inches
        plant_spacing: Space between plants in inches

    Returns:
        int: Number of plants that fit
    """
    try:
        # This could fail if inputs aren't numbers
        length = float(row_length)
        spacing = float(plant_spacing)

        # This could fail with division by zero
        plants = int(length / spacing)
        return plants

    except ValueError:
        print("Error: Inputs must be numbers")
        return 0
    except ZeroDivisionError:
        print("Error: Plant spacing cannot be zero")
        return 0

# Test different error conditions
print(calculate_plants_per_row(48, 12))      # Works: 4
print(calculate_plants_per_row("48", "x"))  # ValueError: 0
print(calculate_plants_per_row(48, 0))       # ZeroDivisionError: 0

4
Error: Inputs must be numbers
0
Error: Plant spacing cannot be zero
0


### Advanced try/except: else and finally

Python provides two additional clauses for more sophisticated error handling:

- **else**: Runs only if NO exception occurred
- **finally**: ALWAYS runs, whether there was an exception or not

**Structure:**
```python
try:
    # Risky code
except SomeError:
    # Handle error
else:
    # Success code (runs if no exception)
finally:
    # Cleanup code (always runs)
```

In [8]:
# Example: Reading garden data from a file
import json

def load_plant_database(filename):
    """Load plant information from JSON file.

    Returns:
        dict: Plant database, or empty dict on error
    """
    print(f"Attempting to load {filename}...")
    file_handle = None

    try:
        file_handle = open(filename, 'r')
        data = json.load(file_handle)

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found")
        return {}

    except json.JSONDecodeError:
        print(f"Error: File '{filename}' is not valid JSON")
        return {}

    else:
        # This only runs if NO exception occurred
        print(f"Successfully loaded {len(data)} plant records")
        return data

    finally:
        # This ALWAYS runs - cleanup
        if file_handle:
            file_handle.close()
            print("File handle closed")

# Demo (will fail gracefully since file doesn't exist)
plants = load_plant_database("plants.json")

Attempting to load plants.json...
Error: File 'plants.json' not found


### Raising Exceptions

Sometimes YOU need to signal that something is wrong. Use the `raise` keyword to create exceptions:

**When to raise exceptions:**
- Input validation fails
- Business rules are violated
- Impossible states are detected

**Syntax:**
```python
raise ExceptionType("Helpful error message")
```

In [9]:
# Example: Validating PlantingContainer dimensions
class PlantingContainer:
    def __init__(self, container_type, length, width, depth=12):
        """Create a new planting container.

        Args:
            container_type: Type like 'bed', 'pot', 'planter'
            length: Length in inches (must be positive)
            width: Width in inches (must be positive)
            depth: Depth in inches (must be positive)

        Raises:
            ValueError: If any dimension is negative or zero
            TypeError: If dimensions aren't numbers
        """
        # Validate types
        if not isinstance(container_type, str):
            raise TypeError("container_type must be a string")

        # Validate dimensions are positive
        if length <= 0:
            raise ValueError(f"length must be positive, got {length}")
        if width <= 0:
            raise ValueError(f"width must be positive, got {width}")
        if depth <= 0:
            raise ValueError(f"depth must be positive, got {depth}")

        self.container_type = container_type
        self.length = length
        self.width = width
        self.depth = depth

    def calculate_volume(self):
        """Calculate soil volume in cubic inches."""
        return self.length * self.width * self.depth

# Valid container
bed1 = PlantingContainer("raised_bed", 48, 24, 8)
print(f"Bed volume: {bed1.calculate_volume()} cubic inches")

# This will raise ValueError
try:
    bad_bed = PlantingContainer("bed", -10, 20, 8)
except ValueError as e:
    print(f"Caught error: {e}")

Bed volume: 9216 cubic inches
Caught error: length must be positive, got -10


### Custom Exception Classes

You can create your own exception types for domain-specific errors. This makes your code more expressive and easier to debug.

**Why use custom exceptions?**
- More descriptive error names
- Can catch specific errors without catching everything
- Professional code organization

**Basic pattern:**
```python
class MyCustomError(Exception):
    """Description of when this error occurs."""
    pass
```

In [10]:
# Garden Management Custom Exceptions

class GardenError(Exception):
    """Base exception for garden management system."""
    pass

class InvalidPlantingDateError(GardenError):
    """Raised when trying to plant at inappropriate time."""
    pass

class IncompatiblePlantsError(GardenError):
    """Raised when plants are incompatible companions."""
    pass

class ContainerFullError(GardenError):
    """Raised when trying to add plants to full container."""
    pass

# Using custom exceptions
def validate_planting_date(plant_type, date, last_frost_date):
    """Check if it's safe to plant.

    Args:
        plant_type: Type of plant (e.g., 'tomato', 'lettuce')
        date: Proposed planting date
        last_frost_date: Last expected frost date

    Raises:
        InvalidPlantingDateError: If planting too early for this plant
    """
    frost_sensitive = ['tomato', 'pepper', 'basil', 'cucumber']

    if plant_type in frost_sensitive:
        if date < last_frost_date:
            raise InvalidPlantingDateError(
                f"Cannot plant {plant_type} before last frost date ({last_frost_date}). "
                f"Risk of frost damage!"
            )

    print(f"✓ Safe to plant {plant_type} on {date}")

# Demo
from datetime import datetime, timedelta

last_frost = datetime(2025, 4, 15)
early_date = datetime(2025, 4, 1)
safe_date = datetime(2025, 5, 1)

# This works
validate_planting_date('lettuce', early_date, last_frost)

# This raises custom exception
try:
    validate_planting_date('tomato', early_date, last_frost)
except InvalidPlantingDateError as e:
    print(f"Caught planting error: {e}")

✓ Safe to plant lettuce on 2025-04-01 00:00:00
Caught planting error: Cannot plant tomato before last frost date (2025-04-15 00:00:00). Risk of frost damage!


### Exception Hierarchy and Catching

**Important:** Exceptions form a hierarchy. You can catch a parent exception to handle multiple child exceptions.

```
GardenError (parent)
    ├── InvalidPlantingDateError
    ├── IncompatiblePlantsError  
    └── ContainerFullError
```

**Best practice:** Order exception handlers from most specific to most general.

In [11]:
def process_planting_request(plant_type, container, date):
    """Process a planting request with comprehensive error handling."""
    try:
        # Various validations that might raise exceptions
        validate_planting_date(plant_type, date, datetime(2025, 4, 15))
        # ... other validations ...
        print(f"Successfully planted {plant_type}")

    except InvalidPlantingDateError as e:
        # Handle specific planting date errors
        print(f"Date error: {e}")
        print("Suggestion: Wait until after last frost")

    except IncompatiblePlantsError as e:
        # Handle companion planting errors
        print(f"Compatibility error: {e}")
        print("Suggestion: Choose different neighboring plants")

    except GardenError as e:
        # Catch any other garden-specific errors
        print(f"Garden system error: {e}")

    except Exception as e:
        # Catch unexpected errors
        print(f"Unexpected error: {e}")
        # In production, you might log this for debugging

# Demo
process_planting_request('tomato', None, datetime(2025, 4, 1))

Date error: Cannot plant tomato before last frost date (2025-04-15 00:00:00). Risk of frost damage!
Suggestion: Wait until after last frost


---
# Part 2: Unit Testing Fundamentals
## (30 minutes)

### Why Testing Matters

**Testing ensures your code actually works!**

**Benefits of automated testing:**
- Catch bugs before users do
- Verify code still works after changes
- Document expected behavior
- Build confidence in your code

**Professional standard:** Code without tests is considered incomplete.

### Python's unittest Framework

Python's built-in `unittest` module provides everything you need for basic testing.

**Basic structure:**
```python
import unittest

class TestSomething(unittest.TestCase):
    def test_feature_name(self):
        # Arrange: Set up test data
        # Act: Do something
        # Assert: Check the result
        self.assertEqual(result, expected)
```

In [12]:
import unittest

# Let's test our PlantingContainer class
class TestPlantingContainer(unittest.TestCase):
    """Tests for PlantingContainer class."""

    def test_volume_calculation(self):
        """Test that volume is calculated correctly."""
        # Arrange: Create a container
        container = PlantingContainer("bed", 48, 24, 8)

        # Act: Calculate volume
        volume = container.calculate_volume()

        # Assert: Check the result
        expected = 48 * 24 * 8  # 9,216 cubic inches
        self.assertEqual(volume, expected)

    def test_negative_length_raises_error(self):
        """Test that negative length raises ValueError."""
        # Assert that creating container with negative length raises error
        with self.assertRaises(ValueError):
            PlantingContainer("bed", -10, 24, 8)

    def test_zero_width_raises_error(self):
        """Test that zero width raises ValueError."""
        with self.assertRaises(ValueError):
            PlantingContainer("bed", 48, 0, 8)

# Run tests (in notebook, we use this pattern)
# In real code files, you'd use: python -m unittest test_file.py
unittest.main(argv=[''], exit=False, verbosity=2)

test_negative_length_raises_error (__main__.TestPlantingContainer.test_negative_length_raises_error)
Test that negative length raises ValueError. ... ok
test_volume_calculation (__main__.TestPlantingContainer.test_volume_calculation)
Test that volume is calculated correctly. ... ok
test_zero_width_raises_error (__main__.TestPlantingContainer.test_zero_width_raises_error)
Test that zero width raises ValueError. ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.024s

OK


### Common Assertion Methods

unittest provides many assertion methods. Here are the most important ones:

| Method | Checks |
|--------|--------|
| `assertEqual(a, b)` | a == b |
| `assertNotEqual(a, b)` | a != b |
| `assertTrue(x)` | bool(x) is True |
| `assertFalse(x)` | bool(x) is False |
| `assertIn(a, b)` | a in b |
| `assertIsNone(x)` | x is None |
| `assertRaises(Exception)` | Code raises that exception |
| `assertAlmostEqual(a, b)` | a ≈ b (for floats) |

In [ ]:
class TestGardenValidation(unittest.TestCase):
    """Tests for garden validation functions."""

    def test_frost_sensitive_plants(self):
        """Test that frost-sensitive plants are correctly identified."""
        frost_sensitive = ['tomato', 'pepper', 'basil', 'cucumber']

        # These should be frost-sensitive
        self.assertIn('tomato', frost_sensitive)
        self.assertIn('pepper', frost_sensitive)

        # These should NOT be frost-sensitive
        self.assertNotIn('lettuce', frost_sensitive)
        self.assertNotIn('carrot', frost_sensitive)

    def test_planting_spacing_calculation(self):
        """Test plant spacing calculations."""
        row_length = 48  # inches
        plant_spacing = 12  # inches

        plants = row_length // plant_spacing

        self.assertEqual(plants, 4)
        self.assertIsInstance(plants, int)
        self.assertTrue(plants > 0)

    def test_soil_volume_float_precision(self):
        """Test that volume calculations handle floats correctly."""
        container = PlantingContainer("pot", 10.5, 10.5, 8.0)
        volume = container.calculate_volume()

        expected = 10.5 * 10.5 * 8.0  # 882.0
        self.assertAlmostEqual(volume, expected, places=2)

# Run tests
unittest.main(argv=[''], exit=False, verbosity=2)

### setUp and tearDown Methods

Often you need to prepare test data before each test. Use `setUp()` for this:

- **setUp()**: Runs BEFORE each test method
- **tearDown()**: Runs AFTER each test method (cleanup)

This is especially useful when multiple tests need similar objects.

In [ ]:
class GardenSystem:
    """Simple garden management system for testing."""
    def __init__(self):
        self.containers = []
        self.plants = []

    def add_container(self, container):
        """Add a container to the system."""
        self.containers.append(container)

    def add_plant(self, plant_name):
        """Add a plant to the system."""
        if not plant_name:
            raise ValueError("Plant name cannot be empty")
        self.plants.append(plant_name)

    def total_soil_volume(self):
        """Calculate total soil volume across all containers."""
        return sum(c.calculate_volume() for c in self.containers)


class TestGardenSystem(unittest.TestCase):
    """Tests for GardenSystem class."""

    def setUp(self):
        """Create a fresh garden system before each test."""
        self.garden = GardenSystem()

        # Add some standard test containers
        self.bed1 = PlantingContainer("bed", 48, 24, 8)
        self.pot1 = PlantingContainer("pot", 12, 12, 10)

        print("\n[setUp] Fresh garden system created")

    def tearDown(self):
        """Clean up after each test."""
        print("[tearDown] Test complete")
        # In real scenarios, might close files, database connections, etc.

    def test_add_container(self):
        """Test adding containers to system."""
        self.garden.add_container(self.bed1)
        self.assertEqual(len(self.garden.containers), 1)

    def test_add_multiple_containers(self):
        """Test adding multiple containers."""
        self.garden.add_container(self.bed1)
        self.garden.add_container(self.pot1)
        self.assertEqual(len(self.garden.containers), 2)

    def test_total_volume(self):
        """Test total soil volume calculation."""
        self.garden.add_container(self.bed1)
        self.garden.add_container(self.pot1)

        expected = self.bed1.calculate_volume() + self.pot1.calculate_volume()
        self.assertEqual(self.garden.total_soil_volume(), expected)

    def test_add_empty_plant_name_raises_error(self):
        """Test that empty plant name raises ValueError."""
        with self.assertRaises(ValueError):
            self.garden.add_plant("")

# Run tests - notice setUp runs before each test!
unittest.main(argv=[''], exit=False, verbosity=2)

### Testing Exception Handling

An important part of testing is verifying that your code raises exceptions when it should.

**Two ways to test exceptions:**

1. **assertRaises as context manager:**
```python
with self.assertRaises(ExceptionType):
    code_that_should_raise_exception()
```

2. **assertRaises as method:**
```python
self.assertRaises(ExceptionType, function, args)
```

In [13]:
class TestGardenExceptions(unittest.TestCase):
    """Test that garden validation raises appropriate exceptions."""

    def test_invalid_planting_date_raises_error(self):
        """Test that early planting raises InvalidPlantingDateError."""
        last_frost = datetime(2025, 4, 15)
        early_date = datetime(2025, 4, 1)

        # Context manager style
        with self.assertRaises(InvalidPlantingDateError):
            validate_planting_date('tomato', early_date, last_frost)

    def test_safe_planting_date_no_error(self):
        """Test that safe planting date doesn't raise error."""
        last_frost = datetime(2025, 4, 15)
        safe_date = datetime(2025, 5, 1)

        # This should NOT raise an exception
        try:
            validate_planting_date('tomato', safe_date, last_frost)
            success = True
        except InvalidPlantingDateError:
            success = False

        self.assertTrue(success)

    def test_error_message_content(self):
        """Test that error message contains helpful information."""
        last_frost = datetime(2025, 4, 15)
        early_date = datetime(2025, 4, 1)

        # Capture the actual exception to check its message
        with self.assertRaises(InvalidPlantingDateError) as context:
            validate_planting_date('tomato', early_date, last_frost)

        # Check that error message is helpful
        error_message = str(context.exception)
        self.assertIn('tomato', error_message)
        self.assertIn('frost', error_message.lower())

# Run tests
unittest.main(argv=[''], exit=False, verbosity=2)

test_error_message_content (__main__.TestGardenExceptions.test_error_message_content)
Test that error message contains helpful information. ... ok
test_invalid_planting_date_raises_error (__main__.TestGardenExceptions.test_invalid_planting_date_raises_error)
Test that early planting raises InvalidPlantingDateError. ... ok
test_safe_planting_date_no_error (__main__.TestGardenExceptions.test_safe_planting_date_no_error)
Test that safe planting date doesn't raise error. ... ok
test_negative_length_raises_error (__main__.TestPlantingContainer.test_negative_length_raises_error)
Test that negative length raises ValueError. ... ok
test_volume_calculation (__main__.TestPlantingContainer.test_volume_calculation)
Test that volume is calculated correctly. ... ok
test_zero_width_raises_error (__main__.TestPlantingContainer.test_zero_width_raises_error)
Test that zero width raises ValueError. ... ok



✓ Safe to plant tomato on 2025-05-01 00:00:00


----------------------------------------------------------------------
Ran 6 tests in 0.291s

OK


### Test Organization Best Practices

**Good test structure:**

1. **One test file per source file**
   - `planting_container.py` → `test_planting_container.py`

2. **One test class per class being tested**
   - Testing `PlantingContainer` → `TestPlantingContainer`

3. **Descriptive test names**
   - ✓ `test_negative_depth_raises_value_error`
   - ✗ `test1`, `test_bad`, `test_error`

4. **Test one thing per test**
   - Each test should verify one specific behavior

5. **AAA pattern: Arrange, Act, Assert**
   - Arrange: Set up test data
   - Act: Execute the code being tested
   - Assert: Verify the results

In [ ]:
class TestPlantingContainerValidation(unittest.TestCase):
    """Tests for PlantingContainer input validation."""

    # Good: Each test focuses on one specific validation

    def test_negative_length_raises_value_error(self):
        """Test that negative length raises ValueError."""
        # Arrange: Set up test data
        negative_length = -10

        # Act & Assert: Verify exception is raised
        with self.assertRaises(ValueError) as context:
            PlantingContainer("bed", negative_length, 24, 8)

        # Optional: Verify error message quality
        self.assertIn("positive", str(context.exception))

    def test_negative_width_raises_value_error(self):
        """Test that negative width raises ValueError."""
        # Separate test for width validation
        with self.assertRaises(ValueError):
            PlantingContainer("bed", 48, -24, 8)

    def test_negative_depth_raises_value_error(self):
        """Test that negative depth raises ValueError."""
        # Separate test for depth validation
        with self.assertRaises(ValueError):
            PlantingContainer("bed", 48, 24, -8)

    def test_non_string_type_raises_type_error(self):
        """Test that non-string container_type raises TypeError."""
        # Test type validation separately
        with self.assertRaises(TypeError):
            PlantingContainer(123, 48, 24, 8)

    def test_valid_container_created_successfully(self):
        """Test that valid inputs create container successfully."""
        # Arrange: Valid inputs
        container_type = "raised_bed"
        length, width, depth = 48, 24, 8

        # Act: Create container
        container = PlantingContainer(container_type, length, width, depth)

        # Assert: Verify attributes set correctly
        self.assertEqual(container.container_type, container_type)
        self.assertEqual(container.length, length)
        self.assertEqual(container.width, width)
        self.assertEqual(container.depth, depth)

# Run tests
unittest.main(argv=[''], exit=False, verbosity=2)

---
# Part 3: Practical Integration
## (10 minutes)

### Complete Example: Garden Validation Framework

Let's put it all together with a complete example showing:
- Custom exceptions
- Comprehensive validation
- Error handling
- Unit tests

In [ ]:
# Complete Garden Validation System

from datetime import datetime, timedelta

# 1. Custom Exceptions
class GardenValidationError(Exception):
    """Base exception for garden validation errors."""
    pass

class InvalidDimensionError(GardenValidationError):
    """Raised when container dimensions are invalid."""
    pass

class InvalidPlantingDateError(GardenValidationError):
    """Raised when planting date is inappropriate."""
    pass

# 2. Validation Functions
def validate_container_dimensions(length, width, depth):
    """Validate container dimensions.

    Args:
        length: Container length in inches
        width: Container width in inches
        depth: Container depth in inches

    Raises:
        InvalidDimensionError: If any dimension is invalid
    """
    try:
        l = float(length)
        w = float(width)
        d = float(depth)
    except (TypeError, ValueError):
        raise InvalidDimensionError("Dimensions must be numeric")

    if l <= 0 or w <= 0 or d <= 0:
        raise InvalidDimensionError("All dimensions must be positive")

    if l > 1000 or w > 1000 or d > 100:
        raise InvalidDimensionError("Dimensions exceed reasonable maximums")

def validate_planting_date_complete(plant_type, date, last_frost, first_frost):
    """Validate planting date for a plant type.

    Args:
        plant_type: Type of plant
        date: Proposed planting date
        last_frost: Last expected spring frost
        first_frost: First expected fall frost

    Raises:
        InvalidPlantingDateError: If planting date is inappropriate
    """
    frost_sensitive = ['tomato', 'pepper', 'basil', 'cucumber', 'melon']

    # Check if date is in the past
    if date < datetime.now():
        raise InvalidPlantingDateError("Cannot plant in the past")

    # Check if too early for frost-sensitive plants
    if plant_type.lower() in frost_sensitive:
        if date < last_frost:
            days_until_safe = (last_frost - date).days
            raise InvalidPlantingDateError(
                f"{plant_type} is frost-sensitive. "
                f"Wait {days_until_safe} more days until {last_frost.date()}"
            )

    # Check if too late in season
    if date > first_frost - timedelta(days=60):  # Need 60 days before frost
        raise InvalidPlantingDateError(
            f"Too late to plant {plant_type}. Not enough time before first frost."
        )

# 3. Safe wrapper function with comprehensive error handling
def create_validated_container(container_type, length, width, depth):
    """Create container with validation and error handling.

    Returns:
        PlantingContainer or None if validation fails
    """
    try:
        # Validate inputs
        validate_container_dimensions(length, width, depth)

        # Create container
        container = PlantingContainer(container_type, length, width, depth)

        print(f"✓ Created {container_type}: {length}×{width}×{depth}")
        return container

    except InvalidDimensionError as e:
        print(f"✗ Dimension error: {e}")
        return None

    except (TypeError, ValueError) as e:
        print(f"✗ Invalid input: {e}")
        return None

    except Exception as e:
        print(f"✗ Unexpected error: {e}")
        return None

# Demo usage
print("=== Creating containers ===")
bed1 = create_validated_container("raised_bed", 48, 24, 8)
bed2 = create_validated_container("pot", -10, 20, 5)  # Invalid
bed3 = create_validated_container("planter", "not a number", 20, 5)  # Invalid

In [ ]:
# 4. Comprehensive Test Suite
class TestGardenValidationFramework(unittest.TestCase):
    """Complete test suite for garden validation."""

    def setUp(self):
        """Set up test dates."""
        self.last_frost = datetime(2025, 4, 15)
        self.first_frost = datetime(2025, 10, 15)
        self.safe_date = datetime(2025, 5, 1)

    # Dimension validation tests
    def test_valid_dimensions_accepted(self):
        """Test that valid dimensions pass validation."""
        # Should not raise any exception
        validate_container_dimensions(48, 24, 8)

    def test_negative_dimension_rejected(self):
        """Test that negative dimensions raise error."""
        with self.assertRaises(InvalidDimensionError):
            validate_container_dimensions(-10, 24, 8)

    def test_zero_dimension_rejected(self):
        """Test that zero dimensions raise error."""
        with self.assertRaises(InvalidDimensionError):
            validate_container_dimensions(48, 0, 8)

    def test_non_numeric_dimension_rejected(self):
        """Test that non-numeric dimensions raise error."""
        with self.assertRaises(InvalidDimensionError):
            validate_container_dimensions("big", 24, 8)

    def test_excessive_dimension_rejected(self):
        """Test that unreasonably large dimensions raise error."""
        with self.assertRaises(InvalidDimensionError):
            validate_container_dimensions(5000, 24, 8)

    # Planting date validation tests
    def test_safe_planting_date_accepted(self):
        """Test that safe planting date is accepted."""
        # Should not raise exception
        validate_planting_date_complete(
            'tomato', self.safe_date, self.last_frost, self.first_frost
        )

    def test_early_frost_sensitive_rejected(self):
        """Test that early planting of frost-sensitive plants is rejected."""
        early_date = datetime(2025, 4, 1)
        with self.assertRaises(InvalidPlantingDateError):
            validate_planting_date_complete(
                'tomato', early_date, self.last_frost, self.first_frost
            )

    def test_late_season_planting_rejected(self):
        """Test that too-late planting is rejected."""
        late_date = datetime(2025, 10, 1)
        with self.assertRaises(InvalidPlantingDateError):
            validate_planting_date_complete(
                'tomato', late_date, self.last_frost, self.first_frost
            )

    # Integration tests
    def test_create_valid_container_succeeds(self):
        """Test that valid container creation succeeds."""
        container = create_validated_container("bed", 48, 24, 8)
        self.assertIsNotNone(container)
        self.assertIsInstance(container, PlantingContainer)

    def test_create_invalid_container_returns_none(self):
        """Test that invalid container creation returns None."""
        container = create_validated_container("bed", -10, 24, 8)
        self.assertIsNone(container)

# Run complete test suite
unittest.main(argv=[''], exit=False, verbosity=2)

### Error Handling Best Practices Summary

**DO:**
- ✓ Use specific exception types (ValueError, not Exception)
- ✓ Provide helpful error messages
- ✓ Create custom exceptions for domain-specific errors
- ✓ Catch exceptions you can handle
- ✓ Let unexpected exceptions propagate
- ✓ Use finally for cleanup code
- ✓ Document what exceptions your functions raise

**DON'T:**
- ✗ Use bare `except:` clauses (catches everything including KeyboardInterrupt!)
- ✗ Silently swallow exceptions without logging
- ✗ Use exceptions for normal control flow
- ✗ Catch exceptions you can't handle
- ✗ Forget to validate inputs
- ✗ Use generic error messages like "Error" or "Something went wrong"

### Testing Best Practices Summary

**DO:**
- ✓ Test both success and failure cases
- ✓ Use descriptive test names
- ✓ Follow AAA pattern (Arrange, Act, Assert)
- ✓ Test one thing per test
- ✓ Use setUp for common test data
- ✓ Test exception handling
- ✓ Run tests frequently during development

**DON'T:**
- ✗ Test only the happy path
- ✗ Write tests that depend on each other
- ✗ Test multiple behaviors in one test
- ✗ Skip testing error conditions
- ✗ Ignore failing tests
- ✗ Copy-paste test code without modifying it

---
# Wrap-up & Preview
## (5 minutes)

### Key Takeaways

1. **Exception handling makes code robust**
   - Use try/except to handle errors gracefully
   - Raise exceptions when validation fails
   - Create custom exceptions for clarity

2. **Testing ensures code quality**
   - Write tests as you develop
   - Test both success and failure cases
   - Use unittest framework for structure

3. **Validation protects your system**
   - Validate inputs early and thoroughly
   - Provide helpful error messages
   - Handle edge cases explicitly

### Week 7 Assignments

**Due Sunday, November 2:**
- **Weekly Discussion 7:** Testing strategies and quality assurance
- **Weekly Exercise 7:** Garden Validation Framework
  - Implement comprehensive error handling for containers
  - Validate planting dates and plant compatibility
  - Create custom exceptions
  - Write unit tests for all validation
- **GitHub: AI Journal 7:** Document AI assistance in testing and debugging
- **GitHub: Repository Action 2:** Testing workflow collaboration
- **Project 2 DUE:** OOP Class Implementation (65 points)

### Next Week Preview: Inheritance

**Week 8 introduces:**
- Single inheritance and class hierarchies
- The `super()` function
- Method overriding
- Creating specialized container classes (Bed, Pot, Planter)

**Project 3 begins:** Advanced OOP with Inheritance & Polymorphism

### Resources

**Python Documentation:**
- [Built-in Exceptions](https://docs.python.org/3/library/exceptions.html)
- [unittest Module](https://docs.python.org/3/library/unittest.html)

**Readings (Week 7):**
- Percival, H. (2017). "Getting started with TDD" from *Test-Driven Development with Python*
- Mollick, E. & Mollick, L. (2023). "Using AI to implement effective teaching strategies" (on AI-assisted debugging)

**Support:**
- TA office hours for testing and debugging help
- Lab sessions for hands-on practice
- Course Slack for quick questions

### Questions?

Remember: Good error handling and testing are marks of professional developers. They make your code more reliable, maintainable, and user-friendly!

---
# Instructor Notes

### Timing Guidance
- **Part 1 (Exception Handling): 35 minutes**
  - Basic try/except: 10 min
  - Raising exceptions: 10 min  
  - Custom exceptions: 10 min
  - Exception hierarchy: 5 min

- **Part 2 (Testing): 30 minutes**
  - unittest basics: 10 min
  - Assertions: 8 min
  - setUp/tearDown: 7 min
  - Testing exceptions: 5 min

- **Part 3 (Integration): 10 minutes**
  - Complete example walkthrough
  - Best practices summary

### Key Teaching Points

1. **Exception handling is defensive programming**
   - Anticipate what can go wrong
   - Handle errors gracefully
   - Provide helpful feedback

2. **Testing gives confidence**
   - Tests document expected behavior
   - Tests catch regressions
   - Tests enable refactoring

3. **Professional code is validated and tested**
   - Input validation prevents bugs
   - Tests are not optional
   - Error messages guide users

### Common Student Struggles

1. **Overly broad exception handling**
   - Remind: Catch specific exceptions
   - Show dangers of bare `except:`

2. **Not understanding when to raise vs. catch**
   - Raise: When YOU detect a problem
   - Catch: When you can HANDLE a problem

3. **Testing feels like extra work**
   - Emphasize: Tests save debugging time
   - Show: How tests catch bugs early

4. **Unclear what to test**
   - Guide: Test both success and failure paths
   - Example: Edge cases and boundary conditions

### Interactive Elements

- Live coding: Show exception handling fixing crash
- Student participation: What could go wrong scenarios
- Quick polls: Testing coverage strategies
- Pair discuss: Error message quality

### Project Connection

This week's concepts directly support:
- Project 2 completion (due this week)
- Project 3 launch (next week)
- All future development work

### Lab Session Focus

- Implementing garden validation framework
- Writing comprehensive tests
- Creating custom exceptions
- Debugging with proper error handling

### Assessment Connection

- Exercise 7: Tests these exact skills
- Project grading includes error handling quality
- Testing will be evaluated in all future projects